## `MyResponseTransformer` Class
Extends `PioResponseTransformer` class.  Must implement the `transform_response()` method.

### Convert `json` -> `dict`

In [ ]:
from pio_model import PioRequestTransformer

class MyRequestTransformer(PioRequestTransformer):        
    def __init__(self, 
                 *args,
                 **kwargs):
        
        PioRequestTransformer.__init__(self, 
                                       args, 
                                       kwargs)

        
    def transform_request(self,
                          request,
                          *args,
                          **kwargs):
        
        request_str = request.decode('utf-8')
        request_str = request_str.strip().replace('\n', ',')
        request_dict = json.loads(request_str)
        return request_dict

## Create Response Transformer
Must have a `transform_response()` method.

### Convert `dict` -> `json`

In [ ]:
from pio_model import PioResponseTransformer

class MyResponseTransformer(PioResponseTransformer):

    def transform_response(self,
                           response):
        response_json = json.dumps(response)
        return response_json

## Train My Model

In [ ]:
## No training is needed.  "Learned Variables" are hard-coded in this example.

## Create Model Initializer
Must implement the `initialize_model()` method

In [ ]:
from pio_model import PioModelInitializer

class MyModelInitializer(PioModelInitializer): 

    def __init__(self,
                 *args,
                 **kwargs):

        PioModelInitializer.__init__(self, 
                                     args, 
                                     kwargs)

        
    def initialize_model(self,                        
                         *args,
                         **kwargs):

        PioModelInitializer.initialize_model(self, 
                                             args, 
                                             kwargs)

        model = {'cat_mean':0.1,
                 'cat_stdv':0.20,
                 'dog_mean':0.3,
                 'dog_stdv':0.40}
        
        return model            

## `MyModel` Class
Extends `PioModel` class.  Must implement `predict()` method.

In [ ]:
from pio_model import PioModel

class MyModel(PioModel):

    def __init__(self, 
                 request_transformer, 
                 response_transformer,
                 model_initializer,
                 *args,
                 **kwargs):

        PioModel.__init__(self,
                          request_transformer,
                          response_transformer,
                          model_initializer,
                          args,
                          kwargs)

    
    def predict(self, 
                request):
        transformed_request = self.request_transformer.transform_request(request)

        cat_affinity_score = sum([ d['weight'] * d['user_score'] for d in transformed_request if 'cat' in d['tags'] ])
        dog_affinity_score = sum([ d['weight'] * d['user_score'] for d in transformed_request if 'dog' in d['tags'] ])

        # create normalized z score for compare/classify
        cat_zscore = (cat_affinity_score - self.model['cat_mean'])/self.model['cat_stdv']
        dog_zscore = (dog_affinity_score - self.model['dog_mean'])/self.model['dog_stdv']

        # classify
        if abs(cat_zscore) > abs(dog_zscore):
            if cat_zscore >= 0:
                category = "cat_lover"
            else:
                category = "cat_hater"
        else:
            if dog_zscore >= 0:
                category = "dog_lover"
            else:
                category = "dog_hater"

        response = {
            'category': category,
            'cat_affinity_score': cat_affinity_score,
            'dog_affinity_score': dog_affinity_score,
            'cat_zscore': cat_zscore,
            'cat_zscore': dog_zscore
        }

        transformed_response = self.response_transformer.transform_response(response)
        return transformed_response

## Construct `MyModel`
Inject Model with Request Transformer, Response Transformer, and Model Initializer
```
MyModel(MyRequestTransformer, 
        MyResponseTransformer,
        MyModelInitializer)
```

In [ ]:
pio_model = MyModel(MyRequestTransformer(),
                    MyResponseTransformer(), 
                    MyModelInitializer())

## Save/Pickle Model as `pio_model.pkl`

In [ ]:
import dill as pickle

pio_model_pkl_path = 'pio_model.pkl'

with open(pio_model_pkl_path, 'wb') as fh:
    pickle.dump(pio_model, fh)

In [ ]:
import subprocess

output = subprocess.check_output('ls -l pio_model.pkl',
                                 stderr=subprocess.STDOUT,
                                 shell=True)

print(output.decode('utf-8'))